# Few-Shotテンプレート

- 著者: [hong-seongmin](https://github.com/hong-seongmin)
- ピアレビュー: [Hye-yoon Jeong](https://github.com/Hye-yoonJeong), [Wooseok Jeong](https://github.com/jeong-wooseok)
- 校正: [BokyungisaGod](https://github.com/BokyungisaGod)
- これは [LangChain OpenTutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial) の一部です

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/02-Prompt/02-FewShotP[...]
[![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/mai[...]

## 概要

LangChain の few-shot プロンプトは、注意深く選ばれた例を与えることで言語モデルが高品質な出力を生成するのを導くための強力なフレームワークを提供します。この手法は必要な明示的な指示の量を最小化し、モデルに望ましい出力の構造やスタイルを学習させやすくします。

- **Few-shot プロンプトテンプレート** : 例を埋め込むことでプロンプトの構造や形式を定義し、モデルが一貫した出力を生成するのを導きます。
- **例の選択戦略** : クエリに対して最も関連性の��い例を動的に選択することで、モデルの文脈理解と応答精度を向上させます。
- **Chroma ベクトルストア** : セマンティック類似性に基づいて例を格納・検索するための強力なユーティリティで、スケーラブルかつ効率的なプロンプト構築を可能にします。

### 目次

- [概要](#overview)
- [環境設定](#environment-setup)
- [FewShotPromptTemplate](#FewShotPromptTemplate)
- [Chroma を用いた動的な例選択](#dynamic-example-selection-with-chroma)
- [FewShotChatMessagePromptTemplate](#FewShotChatMessagePromptTemplate)

### 参考

- [LangChain ドキュメント: Few-shot prompting](https://python.langchain.com/docs/concepts/few_shot_prompting)
- [SQL 質問応答時により良くプロンプトする方法](https://python.langchain.com/docs/how_to/sql_prompting/#few-shot-examples)

---


## 環境設定

環境をセットアップします。詳細については [環境設定](https://wikidocs.net/257836) を参照してください。

**[注意]**
- ```langchain-opentutorial``` はチュートリアルのための使いやすい環境設定、便利な関数、およびユーティリティのセットを提供するパッケージです。
- 詳細は [```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) を確認してください。


In [2]:
%%capture --no-stderr
%pip install langchain-opentutorial

    numpy (>=1.19.*) ; python_version >= "3.7"
           ~~~~~~~^


In [3]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain_openai",
        "langchain-chroma",
    ],
    verbose=False,
    upgrade=False,
)

In [4]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [5]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "09-FewShot-Prompt-Templates",
    }
)

Environment variables have been set successfully.


## ```FewShotPromptTemplate```

Few-shot プロンプティングは、望ましい出力の構造や形式を示す少数の例を提供することで、言語モデルが正確で文脈に合った出力を生成するのを導く強力な手法です。LangChain の `FewShotPromptTemplate` は、この手法を簡単に実装できるようにするためのツールです。

1. **Few-Shot プロンプトの設計**

   - 望ましい出力の構造やスタイルを示す例を定義します。
   - エッジケースを網羅する例を含めることで、モデルの理解と性能を向上させます。

2. **動的な例選択**

   - セマンティック類似度やベクトル検索を使用して、クエリに最も関連する例を選択します。

3. **Few-Shot プロンプトの統合**

   - プロンプトテンプレートと言語モデルを組み合わせて、堅牢なチェーンを作成します。

### ```FewShotPromptTemplate``` の例

`FewShotPromptTemplate` を使うと、モデルに望ましい出力の形式と構造を示す少数の例を与えることができます。以下では、モデルが中間のステップを踏んで質問に答えるのを助けるための例を定義し、その後 `FewShotPromptTemplate` を使ってプロンプトをフォーマットします。

---


In [7]:
from langchain_openai import ChatOpenAI

# Initialize the language model
llm = ChatOpenAI(
    temperature=0,  # Creativity
    model_name="gpt-4o-mini",  # Use a valid model name
)

# User query
question = "What is the capital of United States of America?"

# Query the model
response = llm.predict(question)

# Print the response
print(response)

The capital of the United States of America is Washington, D.C.


In [8]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate

# Define examples for the few-shot prompt
examples = [
    {
        "question": "Who lived longer, Steve Jobs or Einstein?",
        "answer": """Does this question require additional questions: Yes.
Additional Question: At what age did Steve Jobs die?
Intermediate Answer: Steve Jobs died at the age of 56.
Additional Question: At what age did Einstein die?
Intermediate Answer: Einstein died at the age of 76.
The final answer is: Einstein
""",
    },
    {
        "question": "When was the founder of Naver born?",
        "answer": """Does this question require additional questions: Yes.
Additional Question: Who is the founder of Naver?
Intermediate Answer: Naver was founded by Lee Hae-jin.
Additional Question: When was Lee Hae-jin born?
Intermediate Answer: Lee Hae-jin was born on June 22, 1967.
The final answer is: June 22, 1967
""",
    },
    {
        "question": "Who was the reigning king when Yulgok Yi's mother was born?",
        "answer": """Does this question require additional questions: Yes.
Additional Question: Who is Yulgok Yi's mother?
Intermediate Answer: Yulgok Yi's mother is Shin Saimdang.
Additional Question: When was Shin Saimdang born?
Intermediate Answer: Shin Saimdang was born in 1504.
Additional Question: Who was the king of Joseon in 1504?
Intermediate Answer: The king of Joseon in 1504 was Yeonsangun.
The final answer is: Yeonsangun
""",
    },
    {
        "question": "Are the directors of Oldboy and Parasite from the same country?",
        "answer": """Does this question require additional questions: Yes.
Additional Question: Who is the director of Oldboy?
Intermediate Answer: The director of Oldboy is Park Chan-wook.
Additional Question: Which country is Park Chan-wook from?
Intermediate Answer: Park Chan-wook is from South Korea.
Additional Question: Who is the director of Parasite?
Intermediate Answer: The director of Parasite is Bong Joon-ho.
Additional Question: Which country is Bong Joon-ho from?
Intermediate Answer: Bong Joon-ho is from South Korea.
The final answer is: Yes
""",
    },
]

In [9]:
# Create an example prompt template
example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

# Print the first formatted example
print(example_prompt.format(**examples[0]))

Question:
Who lived longer, Steve Jobs or Einstein?
Answer:
Does this question require additional questions: Yes.
Additional Question: At what age did Steve Jobs die?
Intermediate Answer: Steve Jobs died at the age of 56.
Additional Question: At what age did Einstein die?
Intermediate Answer: Einstein died at the age of 76.
The final answer is: Einstein



In [10]:
# Initialize the FewShotPromptTemplate
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

# Example question
question = "How old was Bill Gates when Google was founded?"

# Generate the final prompt
final_prompt = few_shot_prompt.format(question=question)
print(final_prompt)

Question:
Who lived longer, Steve Jobs or Einstein?
Answer:
Does this question require additional questions: Yes.
Additional Question: At what age did Steve Jobs die?
Intermediate Answer: Steve Jobs died at the age of 56.
Additional Question: At what age did Einstein die?
Intermediate Answer: Einstein died at the age of 76.
The final answer is: Einstein


Question:
When was the founder of Naver born?
Answer:
Does this question require additional questions: Yes.
Additional Question: Who is the founder of Naver?
Intermediate Answer: Naver was founded by Lee Hae-jin.
Additional Question: When was Lee Hae-jin born?
Intermediate Answer: Lee Hae-jin was born on June 22, 1967.
The final answer is: June 22, 1967


Question:
Who was the reigning king when Yulgok Yi's mother was born?
Answer:
Does this question require additional questions: Yes.
Additional Question: Who is Yulgok Yi's mother?
Intermediate Answer: Yulgok Yi's mother is Shin Saimdang.
Additional Question: When was Shin Saimdang bo

In [11]:
# Query the model with the final prompt
response = llm.predict(final_prompt)
print(response)

Does this question require additional questions: Yes.  
Additional Question: When was Google founded?  
Intermediate Answer: Google was founded on September 4, 1998.  
Additional Question: What year was Bill Gates born?  
Intermediate Answer: Bill Gates was born on October 28, 1955.  
Additional Question: How old was Bill Gates on September 4, 1998?  
Intermediate Answer: Bill Gates was 42 years old when Google was founded.  
The final answer is: 42 years old.


## Chroma を用いた動的な例選択

時には、単一の質問に答えるために複数の思考ステップを経る必要があります。質問をステップに分解して望ましい答えに導くことで、より高品質な結果が得られることがあります。`Chroma` は、セマンティック類似性に基づいて例を格納および取得する効率的な方法を提供し、ワークフローにおける動的な例選択を可能にします。

1. **埋め込みとベクトルストアの初期化**

   - `OpenAIEmbeddings` を使用して例を埋め込みます。
   - 埋め込みを `Chroma` ベクトルストアに格納して効率的に検索できるようにします。

2. **例の保存**

   - 例は、その内容とメタデータの両方とともに保存されます。
   - メタデータには質問や回答などの詳細を含めることができ、検索後の追加処理に使用されます。

3. **類似性検索**

   - 入力に基づいて最も類似した例をベクトルストアから取得します。
   - 文脈に応じた動的なプロンプト作成を可能にします。


必要なパッケージをインポートし、ベクトルストアを作成します。

In [12]:
from langchain_openai.embeddings.base import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import ChatOpenAI

# Initialize embeddings and vector store
embeddings = OpenAIEmbeddings()
chroma = Chroma(persist_directory="example_selector", embedding_function=embeddings)

回答を導くために必要な例となる質問と回答のプロセスを作成します。

In [13]:
examples = [
    {
        "question": "Who lived longer, Steve Jobs or Einstein?",
        "answer": """Does this question require additional questions: Yes.
Additional Question: At what age did Steve Jobs die?
Intermediate Answer: Steve Jobs died at the age of 56.
Additional Question: At what age did Einstein die?
Intermediate Answer: Einstein died at the age of 76.
The final answer is: Einstein
""",
    },
    {
        "question": "When was the founder of Google born?",
        "answer": """Does this question require additional questions: Yes.
Additional Question: Who is the founder of Google?
Intermediate Answer: Google was founded by Larry Page and Sergey Brin.
Additional Question: When was Larry Page born?
Intermediate Answer: Larry Page was born on March 26, 1973.
Additional Question: When was Sergey Brin born?
Intermediate Answer: Sergey Brin was born on August 21, 1973.
The final answer is: Larry Page was born on March 26, 1973, and Sergey Brin was born on August 21, 1973.
""",
    },
    {
        "question": "Who was the President when Donald Trump's mother was born?",  # 쉼표 추가
        "answer": """Does this question require additional questions: Yes.
Additional Question: Who is Donald Trump's mother?
Intermediate Answer: Donald Trump's mother is Mary Anne MacLeod Trump.
Additional Question: When was Mary Anne MacLeod Trump born?
Intermediate Answer: Mary Anne MacLeod Trump was born on May 10, 1912.
Additional Question: Who was the U.S. President in 1912?
Intermediate Answer: William Howard Taft was President in 1912.
The final answer is: William Howard Taft
""",
    },
    {
        "question": "Are the directors of Oldboy and Parasite from the same country?",
        "answer": """Does this question require additional questions: Yes.
Additional Question: Who is the director of Oldboy?
Intermediate Answer: The director of Oldboy is Park Chan-wook.
Additional Question: Which country is Park Chan-wook from?
Intermediate Answer: Park Chan-wook is from South Korea.
Additional Question: Who is the director of Parasite?
Intermediate Answer: The director of Parasite is Bong Joon-ho.
Additional Question: Which country is Bong Joon-ho from?
Intermediate Answer: Bong Joon-ho is from South Korea.
The final answer is: Yes
""",
    },
]

ベクトルストアを作成し、`DynamicFewShotLearning` テンプレートを定義します。

In [14]:
# Add examples to the vector store
texts = [example["question"] for example in examples]
metadatas = [example for example in examples]
chroma.add_texts(texts=texts, metadatas=metadatas)

# Set up Example Selector
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=chroma,  # Only vectorstore is needed
    k=1  # Number of examples to select
)

# Define Few-Shot Prompt Template
example_prompt_template = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}\n"
)
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt_template,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)


期待するプロセスを通じて回答が得られるかどうかを検証するために実行してみましょう。

In [15]:
# Query input and process
query = {"question": "How old was Elon Musk when he made Paypal?"}
formatted_prompt = prompt.format(**query)

# Print the constructed prompt
print("Constructed Prompt:\n")
print(formatted_prompt)

# Initialize the language model
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Query the language model with the constructed prompt
response = llm.predict(formatted_prompt)
print("\nResponse:\n")
print(response)

Constructed Prompt:

Question:
When was the founder of Google born?
Answer:
Does this question require additional questions: Yes.
Additional Question: Who is the founder of Google?
Intermediate Answer: Google was founded by Larry Page and Sergey Brin.
Additional Question: When was Larry Page born?
Intermediate Answer: Larry Page was born on March 26, 1973.
Additional Question: When was Sergey Brin born?
Intermediate Answer: Sergey Brin was born on August 21, 1973.
The final answer is: Larry Page was born on March 26, 1973, and Sergey Brin was born on August 21, 1973.



Question:
How old was Elon Musk when he made Paypal?
Answer:

Response:

Does this question require additional questions: Yes.  
Additional Question: When was PayPal founded?  
Intermediate Answer: PayPal was founded in December 1998.  
Additional Question: When was Elon Musk born?  
Intermediate Answer: Elon Musk was born on June 28, 1971.  
Additional Question: How old was Elon Musk in December 1998?  
Intermediate An

## ```FewShotChatMessagePromptTemplate```

状況ごとにプロンプトを作成するのは複雑で面倒です。
`FewShotChatMessagePromptTemplate` は、関連する例を組み合わせて構造化された形式でチャットベースのプロンプトを動的に生成するための few-shot 学習アプローチを活用します。この方法は、チャット形式のプロンプト作成を簡素化し、さまざまなユースケースに適用できます。


In [16]:
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_openai.embeddings.base import OpenAIEmbeddings
from langchain_chroma import Chroma

# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings()

# Initialize the Vector DB
chroma = Chroma(persist_directory="fewshot_chat", embedding_function=embeddings)

# Define examples for few-shot prompting : 
examples = [
    {
        "instruction": "You are an expert in writing meeting minutes. Please write meeting minutes based on the given information",
        "input": "On December 25, 2023, XYZ Company's marketing strategy meeting began at 3 PM. The meeting was attended by Marketing Team Leader John Smith, Digital Marketing Manager Sarah Jo[...]
        "answer": """
Meeting Minutes: XYZ Company Marketing Strategy Meeting
Date: December 25, 2023
Location: XYZ Company Conference Room
Attendees: John Smith (Marketing Team Leader), Sarah Johnson (Digital Marketing Manager), Mike Wilson (Social Media Manager)

1. Opening
   - Meeting began with opening remarks by Team Leader John Smith.
   - Meeting purpose: Establish marketing strategies for H1 2024 and discuss new social media campaign ideas.

2. Market Trend Overview (John Smith)
   - Team Leader John Smith presented analysis of recent market trends.
   - Shared insights on consumer behavior changes and competitor strategies.

3. Digital Marketing Strategy (Sarah Johnson)
   - Sarah Johnson presented digital marketing strategy.
   - Focused on online advertising and SEO optimization methods.

4. Social Media Campaign (Mike Wilson)
   - Mike Wilson proposed ideas for new social media campaigns.
   - Explained plans for influencer marketing and content strategy.

5. Comprehensive Discussion
   - Team members shared and discussed ideas.
   - Discussed budget and resource allocation for each strategy.

6. Closing
   - Confirmed next meeting date and time.
   - Sarah Johnson assigned to compile and distribute meeting minutes.
""",
    },
    {
        "instruction": "You are a summarization expert. Please summarize the content based on the given information",
        "input": "This document is a 20-page report on 'Strategies for Sustainable Urban Development'. The report comprehensively covers the importance of sustainable urban development, curren[...]
        "answer": """
Document Summary: Strategy Report for Sustainable Urban Development

- Importance: Emphasizes the necessity of sustainable urban development and its social, economic, and environmental benefits.
- Current Issues: Analyzes major problems in current urbanization processes, such as environmental pollution, resource depletion, and increasing inequality.
- Strategies: Presents various strategies to achieve sustainable urban development. These include eco-friendly construction, public transportation improvements, energy efficiency enhancement, and[...]
- Case Studies: Introduces successful sustainable development cases from cities worldwide. For example, explains achievable strategies through cases like Copenhagen, Denmark and Yokohama, Japan.\[...]
- Lessons: Summarizes key lessons learned from these cases. Emphasized lessons include the importance of multi-faceted approaches, cooperation with local communities, and the need for long-term p[...]

This report provides an in-depth analysis of how sustainable urban development can be realized in practical and effective forms.
""",
    },
    {
        "instruction": "You are a sentence correction expert. Please correct the following sentences",
        "input": "Our company is planning to introduce a new marketing strategy. Through this, communication with customers will become more effective.",
        "answer": "This company expects to improve customer communication more effectively by introducing a new marketing strategy.",
    },
]

In [17]:
# Add examples to the vector store
texts = [example["instruction"] + " " + example["input"] for example in examples]
metadatas = examples
chroma.add_texts(texts=texts, metadatas=metadatas)

# Example query
query = {
    "instruction": "Please write the meeting minutes",
    "input": "On December 26, 2023, the product development team of ABC Technology Company held a weekly progress meeting for a new mobile application project. The meeting was attended by Proj[...]
}

# Perform similarity search
query_text = query["instruction"] + " " + query["input"]
results = chroma.similarity_search(query_text, k=1)
print(results)

[Document(metadata={'answer': '\nMeeting Minutes: XYZ Company Marketing Strategy Meeting\nDate: December 25, 2023\nLocation: XYZ Company Conference Room\nAttendees: John Smith (Marketing Te[...]


In [18]:
# Print the most similar example
print(f"Most similar example to the input:\n{query_text}\n")
for result in results:
    print(f'Instruction:\n{result.metadata["instruction"]}')
    print(f'Input:\n{result.metadata["input"]}')
    print(f'Answer:\n{result.metadata["answer"]}')

Most similar example to the input:
Please write the meeting minutes On December 26, 2023, the product development team of ABC Technology Company held a weekly progress meeting for a new mobile application project. The meeting wa[...]

Instruction:
You are an expert in writing meeting minutes. Please write meeting minutes based on the given information
Input:
On December 25, 2023, XYZ Company's marketing strategy meeting began at 3 PM. The meeting was attended by Marketing Team Leader John Smith, Digital Marketing Manager Sarah Johnson, and Social M[...]
Answer:

Meeting Minutes: XYZ Company Marketing Strategy Meeting
Date: December 25, 2023
Location: XYZ Company Conference Room
Attendees: John Smith (Marketing Team Leader), Sarah Johnson (Digital Marketing Manager), Mike Wilson (Social Media Manager)

1. Opening
   - Meeting began with opening remarks by Team Leader John Smith.
   - Meeting purpose: Establish marketing strategies for H1 2024 and discuss new social media campaign ideas.

In [19]:
# Create the final prompt template
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        (
            "human",
            "{instruction}:\n{input}",
        ),
    ]
)

# Combine the prompt and the best example
best_example = results[0].metadata
filled_prompt = final_prompt.format_messages(**best_example)

# Print the filled prompt
print("\nFilled Prompt:\n")
for message in filled_prompt:
    # Determine message type and extract content
    message_type = type(message).__name__  # e.g., SystemMessage, HumanMessage, AIMessage
    content = message.content
    print(f"{message_type}:\n{content}\n")


Filled Prompt:

SystemMessage:
You are a helpful assistant.

HumanMessage:
You are an expert in writing meeting minutes. Please write meeting minutes based on the given information:
On December 25, 2023, XYZ Company's marketing strategy meeting began at 3 PM. The meeting was attended by Marketing Team Leader John Smith, Digital Marketing Manager Sarah Jo[...]



In [20]:
# Query the model with the filled prompt
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
formatted_prompt = "\n".join([f"{type(message).__name__}:\n{message.content}" for message in filled_prompt])
response = llm.predict(formatted_prompt)

# Print the model's response
print("\nResponse:\n")
print(response)


Response:

**Meeting Minutes**

**Date:** December 25, 2023  
**Time:** 3:00 PM  
**Location:** XYZ Company Conference Room  

**Attendees:**  
- John Smith, Marketing Team Leader  
- Sarah Johnson, Digital Marketing Manager  
- Mike Wilson, Social Media Manager  

**Agenda:**  
1. Overview of recent market trends  
2. Discussion of marketing strategies for the first half of 2024  
3. Ideas for new social media campaigns  

**Minutes:**  

1. **Call to Order:**  
   The meeting was called to order by John Smith at 3:00 PM.

2. **Overview of Recent Market Trends:**  
   - John Smith provided a brief overview of the current market trends affecting the industry. He highlighted key insights that will inform the marketing strategies for the upcoming year.

3. **Presentations on Strategic Ideas:**  
   - **Sarah Johnson (Digital Marketing Manager):**  
     - Presented her strategic ideas focusing on enhancing digital presence through targeted online advertising and SEO optimization.  
    

### Example Selector における類似性検索の問題の解決

類似性を計算する際、`instruction` と `input` の両方が使用されます。しかし、`instruction` のみで検索を行うと、正確な類似性結果が得られない場合があります。

これを解決するために、類似性計算のためのカスタムクラスを定義します。

以下は誤って取得された結果の例です。


In [21]:
from langchain_openai.embeddings.base import OpenAIEmbeddings
from langchain_chroma import Chroma

# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings()

# Initialize the Vector DB
chroma = Chroma(persist_directory="fewshot_chat", embedding_function=embeddings)

# Initialize the rest of the workflow...
[...]
